## Story Cloze: Bert for Sequence Classification

#### install tools

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import datasets
from datasets import load_dataset, load_metric
from datasets import Dataset


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# load dataset
dataset = pd.read_json('/content/drive/MyDrive/Project/ROCStories_winter2017_mixed.json')

### Modify Training Test and Validation Datasets

In [7]:
def modify_training(data):
  new = pd.DataFrame()
  new['stories'] = data.story
  new['ending0'] = data.ending0
  new['ending1'] = data.ending1
  new['labels'] = data.label
  ntest = new.melt(id_vars=['stories','labels'], value_vars=['ending0', 'ending1'], var_name='ending_number', value_name='endings')
  check_last_index_match = lambda row: 1 if row['ending_number'][-1] == str(row['labels']) else 0
  ntest['label']=ntest.apply(check_last_index_match, axis=1)
  ntest.drop(['labels','ending_number'],inplace=True,axis=1)
  return ntest.sample(frac=1,replace=False).reset_index(drop=True)

In [8]:
data_mod = modify_training(dataset)
train_df = data_mod[:70000]
validation_df = data_mod[70000:85000]
test_df = data_mod[85000:]

In [9]:
print('Maximum Num Training Words:', max([len(x.split()) for x in train_df.stories]) + max([len(x.split()) for x in train_df.endings]) )

Maximum Num Training Words: 81


In [10]:
### turn all train, validation, and test datasets into a dataset dictionary ###
train_dataset = Dataset.from_dict(train_df)
val_dataset = Dataset.from_dict(validation_df)
test_dataset = Dataset.from_dict(test_df)
data_dict = datasets.DatasetDict({"train":train_dataset,
                                        "validation" : val_dataset,
                                        "test":test_dataset})

## Look at some examples

In [11]:
def show_one(example):
    print(f"Story: {example['stories']}")
    print(f"\nEnding: {example['endings']}")
    print(f"\nLabel: {example['label']}")

In [12]:
show_one(data_dict['train'][999])

Story: My son does his homework every night. He always puts his completed homework in his folder. Yesterday, it was not in his folder when he arrived in class. We looked everywhere and finally found it in his sister's folder.

Ending: One day, even I thought he was cursing, and I stopped laughing at her.

Label: 0


In [13]:
show_one(data_dict['validation'][999])

Story: Debbie got a new cat. She went to the pound to find one. They had so many there. She picked this cute orange one.

Ending: She named her Tabby.

Label: 1


## Preprocess

In [14]:
### load tokenizer ###
from transformers import AutoTokenizer
checkpoint = '/content/drive/MyDrive/Project/bert_model_weights'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

#### Tokenize each sentence

In [15]:
MAX_SEQUENCE_LENGTH = 85
def preprocess_function(examples):
    return tokenizer(examples['stories'], examples['endings'], max_length=MAX_SEQUENCE_LENGTH,
                     truncation=True, padding='max_length',return_tensors='tf')

In [16]:
pre_tokenizer_columns = set(data_dict["train"].features)
encoded_dataset = data_dict.map(preprocess_function, batched=True)
tokenizer_columns = list(set(encoded_dataset["train"].features) - pre_tokenizer_columns)
print("Columns added by tokenizer:", tokenizer_columns)

Map:   0%|          | 0/70000 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/20330 [00:00<?, ? examples/s]

Columns added by tokenizer: ['token_type_ids', 'input_ids', 'attention_mask']


#### Look at some tokenization and preprocessing outputs

In [17]:
examples = data_dict["train"][:3]
features = preprocess_function(examples)
print(
    len(features["input_ids"]),
    len(features["input_ids"][0]),
    [len(x) for x in features["input_ids"]],
)

3 85 [85, 85, 85]


In [18]:
[tokenizer.decode(features["input_ids"][x]) for x in range(len(features))]

['[CLS] james was walking out his front door. his front step was unstable. james stepped on the step and it broke. his foot went through the step. [SEP] james fractured his ankle. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] andrea got on the train. she was headed into the city! she went to museums and did some shopping. she loved the hustle and bustle. [SEP] she was sad to go back home. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] tim liked to play pool. he challenged sam to a game. sam agreed to the challenge. 

## Load Model

In [21]:
from transformers import TFBertForSequenceClassification

id2label = {0: "Invalid", 1: "Valid"}
label2id = {val: key for key, val in id2label.items()}

model = TFBertForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
model.config.dropout = 0.3
model.config.hidden_dropout_prob = 0.3
model.config.attention_probs_dropout_prob = 0.3

In [25]:
tf_train_dataset = model.prepare_tf_dataset(
    encoded_dataset["train"],
    shuffle=True,
    batch_size=16,
    tokenizer=tokenizer
)

tf_validation_dataset = model.prepare_tf_dataset(
    encoded_dataset["validation"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

tf_test_dataset = model.prepare_tf_dataset(
    encoded_dataset["test"],
    shuffle=False,
    batch_size=16,
    tokenizer=tokenizer,
)

Evaluation Metrics

In [26]:
import evaluate
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [30]:
from transformers import create_optimizer

num_epochs = 3
batch_size = 16
batches_per_epoch = len(encoded_dataset["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)

optimizer, schedule = create_optimizer(
    init_lr=3e-05, num_warmup_steps=0, num_train_steps=total_train_steps
)
model.compile(optimizer=optimizer)

In [31]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=tf_validation_dataset
)

In [32]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 109483778 (417.65 MB)
Trainable params: 109483778 (417.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    callbacks=[metric_callback],
)

Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
4375/4375 [==============================] - 1694s 368ms/step - loss: 0.1105 - val_loss: 0.1003 - accuracy: 0.9614
Epoch 2/3
4375/4375 [==============================] - 1555s 355ms/step - loss: 0.0304 - val_loss: 0.1164 - accuracy: 0.9653
Epoch 3/3
4375/4375 [==============================] - 1514s 346ms/step - loss: 0.0070 - val_loss: 0.1440 - accuracy: 0.9652


In [92]:
model.evaluate(tf_test_dataset)

1271/1271 [==============================] - 142s 112ms/step - loss: 0.1333


0.1333094835281372